In [1]:
import os
from PIL import Image
from lang_sam import LangSAM
import matplotlib.pyplot as plt
import numpy as np
import cv2


C:\Users\supha\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
C:\Users\supha\AppData\Local\Programs\Python\Python310\lib\site-packages\groundingdino\models\GroundingDINO\ms_deform_attn.py:31: UserWarning: Failed to load custom C++ ops. Running on CPU mode Only!
  warnings.warn("Failed to load custom C++ ops. Running on CPU mode Only!")


In [2]:
model = LangSAM()


final text_encoder_type: bert-base-uncased


Model loaded from C:\Users\supha\.cache\huggingface\hub\models--ShilongLiu--GroundingDINO\snapshots\a94c9b567a2a374598f05c584e96798a170c56fb\groundingdino_swinb_cogcoor.pth 
 => _IncompatibleKeys(missing_keys=[], unexpected_keys=['label_enc.weight', 'bert.embeddings.position_ids'])


In [3]:
folder_path = r"C:\Users\supha\Documents\65011048\test-image"

In [4]:
text_prompt = "hand, cup, straw"                                 

In [ ]:
# วนลูปผ่านไฟล์ทั้งหมดในโฟลเดอร์
for index_filename, filename in enumerate(os.listdir(folder_path)):
    if filename.endswith(".jpg") or filename.endswith(".png"):  # ตรวจสอบเฉพาะไฟล์ภาพ
        # สร้าง path สำหรับไฟล์ภาพแต่ละไฟล์
        image_path = os.path.join(folder_path, filename)

        # ลดขนาดภาพให้มีความกว้างและสูงที่เล็กลง
        resize_width = 225
        resize_height = 400
        
        # เปิดภาพด้วย PIL
        # image_pil = Image.open(image_path).convert("RGB").resize((resize_width, resize_height))
        image_pil = Image.open(image_path).convert("RGB")

        # ใช้โมเดล LangSAM เพื่อทำการพยากรณ์
        masks, boxes, phrases, logits = model.predict(image_pil, text_prompt)

        
        def calculate_iou(box1, box2):
            x1_max = max(box1[0], box2[0])
            y1_max = max(box1[1], box2[1])
            x2_min = min(box1[2], box2[2])
            y2_min = min(box1[3], box2[3])
    
    
            inter_area = max(0, x2_min - x1_max) * max(0, y2_min - y1_max)
            
            
            box1_area = (box1[2] - box1[0]) * (box1[3] - box1[1])
            box2_area = (box2[2] - box2[0]) * (box2[3] - box2[1])
            
            
            iou = inter_area / float(box1_area + box2_area - inter_area)
            return iou


        iou_threshold = 0.5  
        selected_boxes = []
        selected_masks = []
        selected_logits = []
        selected_phrases = []


        for i, box in enumerate(boxes):
            keep = True
            for j in range(len(selected_boxes)):
                iou = calculate_iou(box, selected_boxes[j])
                if iou > iou_threshold:
                    if logits[i] > selected_logits[j]:  
                        selected_boxes[j] = box
                        selected_masks[j] = masks[i]
                        selected_logits[j] = logits[i]
                        selected_phrases[j] = phrases[i]
                    keep = False
                    break
            if keep:
                selected_boxes.append(box)
                selected_masks.append(masks[i])
                selected_logits.append(logits[i])
                selected_phrases.append(phrases[i])

        masks_np = np.array([mask.detach().cpu().numpy() for mask in selected_masks])
 


        # แปลงภาพ PIL เป็น NumPy array
        image_np = np.array(image_pil).copy()
        
        # สร้างภาพที่มี mask ทั้งหมด
        image_with_all_masks = image_np.copy()
        
        
        # วาดขอบ mask ของแต่ละวัตถุลงในภาพ
        for i, mask in enumerate(selected_masks):
            # กำหนดสีที่ต่างกันสำหรับแต่ละวัตถุ
            if selected_phrases[i] == 'hand':
                mask_color = [0, 0, 255]  # สีน้ำเงิน
            elif selected_phrases[i] == 'cup':
                mask_color = [0, 255, 0]  # สีเขียว
            elif selected_phrases[i] == 'straw':
                mask_color = [255, 0, 0]  # สีแดง
            else:
                mask_color = [0, 255, 255]  # สีม่วง
        
            # แปลง mask ให้เป็นรูปแบบที่ OpenCV รองรับ (uint8)
            mask_uint8 = (masks_np[i] * 255).astype(np.uint8)
        
            # หาขอบของ mask ด้วย OpenCV
            contours, _ = cv2.findContours(mask_uint8, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        
            # วาดขอบของ mask บนภาพต้นฉบับ
            cv2.drawContours(image_with_all_masks, contours, -1, mask_color, thickness=12)  # กำหนดความหนาของขอบได้ด้วย 'thickness'
        
        # แสดงภาพพร้อมขอบ mask ทั้งหมด
        plt.imshow(image_with_all_masks)
        plt.axis('off')
        plt.title(f"Image: {index_filename + 1} / {len(os.listdir(folder_path))}")

        
        # # วาด bounding box ของแต่ละวัตถุด้วยสีที่ตรงกับ mask
        # for i, box in enumerate(selected_boxes):
        #     # แปลงสี mask จาก [0-255] ไปเป็น [0-1] สำหรับการวาด bounding box
        #     box_color = [c / 255 for c in mask_color]
        
        #     x1, y1, x2, y2 = box
        #     plt.gca().add_patch(plt.Rectangle((x1, y1), x2 - x1, y2 - y1, fill=False, color=box_color, linewidth=3))
        
        # แสดงผลลัพธ์สุดท้าย
        plt.show()